# Import libraries

## TODO: move to requirements.txt

In [1]:
!pip install stop-words
!pip install wordcloud
!pip install stanza
!pip install spacy-stanza

You should consider upgrading via the 'c:\users\saox\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\saox\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\saox\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\saox\anaconda3\python.exe -m pip install --upgrade pip' command.


In [ ]:
import nltk
nltk.download('stopwords')
# Download the stanza model if necessary
stanza.download("es")

# Initialize the pipeline
nlp = spacy_stanza.load_pipeline("es")

In [3]:
import pandas as pd
import numpy as np
import re
import unidecode
from nltk.probability import FreqDist
from nltk.corpus import stopwords as swords
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import stanza
import spacy_stanza
import itertools

# Load the dataset

In [4]:
df = pd.read_pickle('initiatives.pkl') 

In [5]:
df.head()

,_id,author_parliamentarygroups,created,initiative_type,initiative_type_alt,reference,status,tagged,title,updated,url,tags,topics,history,author_others,place,content,extra,author_deputies
0,8340a50c3e322491545cf6c40b0b3d29ed03cb85,[Grupo Parlamentario Ciudadanos],{'$date': '2020-06-02T00:00:00Z'},84,Moción de reprobación a miembros del Gobierno,084/000004,Desconocida,True,"Reprobación del Ministro del Interior, don Fer...",{'$date': '2020-06-09T00:00:00Z'},https://www.congreso.es/web/guest/indice-de-in...,[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,06709837ef704e5aaacfa6ea400517c560ebdd53,[Grupo Parlamentario Republicano],{'$date': '2021-03-18T00:00:00Z'},84,Moción de reprobación a miembros del Gobierno,084/000022,Desconocida,True,Reprobación de la Ministra de Defensa.,{'$date': '2021-03-23T00:00:00Z'},https://www.congreso.es/web/guest/indice-de-in...,[],[],NaN,NaN,NaN,NaN,NaN,NaN
2,08cee188414ee529ca4dc998340791bae832029f,[Grupo Parlamentario Popular],{'$date': '2021-03-25T00:00:00Z'},84,Moción de reprobación a miembros del Gobierno,084/000023,Desconocida,True,Reprobación de la Ministra de Educación y Form...,{'$date': '2021-04-06T00:00:00Z'},https://www.congreso.es/web/guest/indice-de-in...,[],[],NaN,NaN,NaN,NaN,NaN,NaN
3,70e6ca135dfb404955c15cf109c2a3614951528f,[Grupo Parlamentario Ciudadanos],{'$date': '2021-02-08T00:00:00Z'},84,Moción de reprobación a miembros del Gobierno,084/000021,Desconocida,True,Reprobación del Vicepresidente Segundo del Gob...,{'$date': '2021-02-16T00:00:00Z'},https://www.congreso.es/web/guest/indice-de-in...,[],[],NaN,NaN,NaN,NaN,NaN,NaN
4,48643b81ce1cd97cbe74ceb711eaafb7ead9241f,[Grupo Parlamentario Popular],{'$date': '2020-06-02T00:00:00Z'},84,Moción de reprobación a miembros del Gobierno,084/000003,Desconocida,True,Reprobación del Ministro del Interior.,{'$date': '2020-06-09T00:00:00Z'},https://www.congreso.es/web/guest/indice-de-in...,[],[],NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# If content is not present, use the title as content
df['content_coalesce'] = df['content'].combine_first(df['title'])

In [7]:
# test
df[df['content'].isna()][['content_coalesce','title','content']]

,content_coalesce,title,content
0,"Reprobación del Ministro del Interior, don Fer...","Reprobación del Ministro del Interior, don Fer...",NaN
1,Reprobación de la Ministra de Defensa.,Reprobación de la Ministra de Defensa.,NaN
2,Reprobación de la Ministra de Educación y Form...,Reprobación de la Ministra de Educación y Form...,NaN
3,Reprobación del Vicepresidente Segundo del Gob...,Reprobación del Vicepresidente Segundo del Gob...,NaN
4,Reprobación del Ministro del Interior.,Reprobación del Ministro del Interior.,NaN
...,...,...,...
110187,Recurso de inconstitucionalidad número 2379/20...,Recurso de inconstitucionalidad número 2379/20...,NaN
110188,Sentencia dictada por el citado Tribunal en el...,Sentencia dictada por el citado Tribunal en el...,NaN
110189,Cuestión de inconstitucionalidad número 3523/2...,Cuestión de inconstitucionalidad número 3523/2...,NaN
110190,Informe de Gestión del ejercicio 2020.,Informe de Gestión del ejercicio 2020.,NaN


In [8]:
# Here we flatten and then create lists of the individual words
# df['lists_content_coalesce'] = [''.join(l).split(" ") for l in df['content_coalesce']]

In [9]:
# THIS FLATTENS
df['content_coalesce'] = [''.join(l) for l in df['content_coalesce']]

# Define helper functions

In [10]:
def create_df_from_json(file_path, columns_to_keep=['content','title','initiative_type_alt'],field_name='initiatives'):
    with open(file_path,'r', encoding="utf8") as f:
        data = json.loads(f.read())
    data_frame = pd.json_normalize(data,record_path=field_name)
    data_frame = data_frame[columns_to_keep]
    return(data_frame)

def retrieve_stop_words():
    spanish_stopwords = swords.words('spanish')
    stop_words_spanish = get_stop_words('spanish')
    stopwords = list(set(spanish_stopwords + stop_words_spanish))
    return stopwords

def space_out_your_text(row):
    doc = nlp(row)
    cleaned = ""
    for token in doc:
        if token.pos_ not in ("PUNCT","ADP","SCONJ","PRON","CCONJ"):
            #print(token.text, token.lemma_, token.pos_, token.dep_)
            cleaned+=token.lemma_+" "
    return cleaned


def remove_accents(row,column):
    return unidecode.unidecode(row[column])

#remove special characters
def replace_special_char(row):
    for word, initial in {".":" ", "-":" ","/":" ","@":" ","#":" ","(":" ",")":" ",'"' : ""}.items():
        row = row.replace(word, initial) 
    return row

def remove_stopwords(row, stopwords):
    removed_stopwords = " ".join([word for word in row.split(" ") if word not in stopwords and word.replace(" ","")!=""])
    return removed_stopwords


def remove_numbers(col):
    return col.str.replace('\d+', '')


def unique_words(col):
    col.str.lower().str.findall("\w+")
    unique = set()

    for x in words:
        unique.update(x)
    return unique


def word_count(df):
    tf = df['text'].apply(lambda x: FreqDist(x)).sum(axis = 0)
    tf2 = dict(tf)
    data_items = tf2.items()
    data_list = list(data_items)
    freq_dataframe = pd.DataFrame(data_list)
    freq_dataframe.columns = ['Word','Counts']
    freq_dataframe = freq_dataframe.sort_values(by="Counts",ascending=False)
    pd.set_option("max_rows", None)
    return freq_dataframe

# Get stopwords

In [11]:
stop_words = retrieve_stop_words()

# Demo Wordcloud

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# example usage: test_df = create_df_from_json('./small-batch.json')
test_df = create_df_from_json('small-batch.json') 

grouped = test_df.groupby("initiative_type_alt")['content'].apply(lambda tags: ','.join(tags))

def show_cloud(i):
    text = grouped[i]
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
    plt.figure(figsize=(12,5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

for i, row in grouped.iteritems():
    unique_id = i   
    print(i)    
    if len(grouped[i])>0:
        show_cloud(i) 

# Apply transformations one-by-one

In [13]:
unique = unique_words(df['content_coalesce'])
print("Nr of words: " + str(len(list(unique))))

NameError: name 'words' is not defined

In [12]:
df['t1_no_accents'] = df.apply(lambda row:remove_accents(row,'content_coalesce'),axis=1)
df['t1_no_accents'].head()

0    Reprobacion del Ministro del Interior, don Fer...
1               Reprobacion de la Ministra de Defensa.
2    Reprobacion de la Ministra de Educacion y Form...
3    Reprobacion del Vicepresidente Segundo del Gob...
4               Reprobacion del Ministro del Interior.
Name: t1_no_accents, dtype: object

In [14]:
df['t2_no_numbers'] = remove_numbers(df['t1_no_accents']) 
df['t2_no_numbers'].head()

0    Reprobacion del Ministro del Interior, don Fer...
1               Reprobacion de la Ministra de Defensa.
2    Reprobacion de la Ministra de Educacion y Form...
3    Reprobacion del Vicepresidente Segundo del Gob...
4               Reprobacion del Ministro del Interior.
Name: t2_no_numbers, dtype: object

In [15]:
df['t3_no_special_char'] = df['t2_no_numbers'].apply(lambda row:replace_special_char(row))
df['t3_no_special_char'].head()

0    Reprobacion del Ministro del Interior, don Fer...
1               Reprobacion de la Ministra de Defensa 
2    Reprobacion de la Ministra de Educacion y Form...
3    Reprobacion del Vicepresidente Segundo del Gob...
4               Reprobacion del Ministro del Interior 
Name: t3_no_special_char, dtype: object

In [16]:
df['t4_lowercase'] = df['t3_no_special_char'].str.lower()
df['t4_lowercase'].head()

0    reprobacion del ministro del interior, don fer...
1               reprobacion de la ministra de defensa 
2    reprobacion de la ministra de educacion y form...
3    reprobacion del vicepresidente segundo del gob...
4               reprobacion del ministro del interior 
Name: t4_lowercase, dtype: object

In [ ]:
df["unigrams"] = df["t4_lowercase"].apply(nltk.word_tokenize)

In [ ]:
df["unigrams"].head()

In [ ]:
df['t5_stopwords_removed'] = df['t4_lowercase'].apply(lambda row:remove_stopwords(row, stop_words))
df['t5_stopwords_removed'].head()

In [ ]:
def lemmatize(text):
    return " ".join([tok.lemma_ for tok in nlp.tokenizer(text) if not tok.is_stop])

df['t6_lemmitization'] = df['t5_stopwords_removed'].apply(lambda x:lemmatize(x))
df['t6_lemmitization'].head()

In [ ]:
df['t5_stopwords_removed'].head()

In [ ]:
unique = unique_words(df['t3_no_special_char'])
print("Nr of words: " + str(len(list(unique))))

In [ ]:
unique = unique_words(df['t5_stopwords_removed'])
print("Nr of words: " + str(len(list(unique))))

In [ ]:
unique = unique_words(df['t6_lemmitization'])
print("Nr of words: " + str(len(list(unique))))

In [ ]:
df[df._id == "8139ec1c206e10ba04dde86d3e06f2698e34b0a6"]['content_coalesce'].values

In [ ]:
# def transform(df):
#     df['content_coalesce'] = df.apply(lambda row:remove_accents(row,'content_coalesce'),axis=1)
# a = transform(df)
# a['content_coalesce'].head()

In [ ]:
#remove punctuations, tabs, etc 
df.apply(lambda row:space_out_your_text(row['lowered']),axis=1)
#Lower case
df.apply(lambda row: row['text'].lower(), axis=1)
df['removed_num'] = df.apply(lambda row: remove_numbers(row['text']), axis=1)    

In [ ]:
df2['tokenized_sents'] = df2.apply(lambda row: nlp(row['removed_num']), axis=1)
df2['tokenized_sents_str'] = df2.tokenized_sents.apply(lambda x:str(x))

infreq = freq_dataframe[freq_dataframe['Counts'] < 3 ]['Word'].tolist()


df2['removed_infreq'] = df2.tokenized_sents_str.apply(lambda x: remove_stopwords(x,infreq))
df2['removed_infreq_str'] = df2.removed_infreq.apply(lambda x:str(x))



v = TfidfVectorizer()
x = v.fit_transform(df2['removed_infreq_str'])
x.todense()
len(v.vocabulary_)